In [ ]:
import pandas as pd

In [ ]:
def get_cross_feature(df, user_id='mobile', item_id='skuid', sequence_type='view', cycle=7):
    """计算用户-商品交叉特征的优化版本

    Args:
        df: 输入数据集
        user_id: 用户ID列名
        item_id: 商品ID列名
        sequence_type: 行为类型 - 'view'/'click'/'purchase'
        cycle: 统计周期
    """
    # 获取对应的行为序列列名
    seq_col_map = {
        'view': 'qysc_view_seq',
        'click': 'qysc_clk_seq',
        'purchase': 'qysc_order_seq'
    }
    sequence_col = seq_col_map[sequence_type]

    # 检查序列是否都相同
    df[sequence_col] = df[sequence_col].apply(lambda x: '[]' if (x is None or len(x) == 0) else str(x))
    first_seq = df[sequence_col].iloc[0]
    # 如果序列为空，直接返回
    if first_seq == '[]' and df[sequence_col].eq(first_seq).all():
        # 如果都是空列表，直接返回带有零值特征的DataFrame
        result_arrays = {
            f'u2i_{cycle}days_{sequence_type}_count': np.zeros(len(df)),
            f'u2i_type1_{cycle}days_{sequence_type}_count': np.zeros(len(df)),
            f'u2i_type2_{cycle}days_{sequence_type}_count': np.zeros(len(df))
        }
        result_df = pd.DataFrame(result_arrays)
        result_df.index = df.index
        return pd.concat([df, result_df], axis=1)
        # 进入优化逻辑
    if df[sequence_col].eq(first_seq).all():
        # 所有序列都相同，只需计算一次
        sequence = eval(first_seq)

        # 预分配结果数组
        result_arrays = {
            f'u2i_{cycle}days_{sequence_type}_count': np.zeros(len(df)),
            f'u2i_type1_{cycle}days_{sequence_type}_count': np.zeros(len(df)),
            f'u2i_type2_{cycle}days_{sequence_type}_count': np.zeros(len(df))
        }

        # 对每个时间点和商品进行向量化计算
        for idx, (current_time, current_item, type1, type2) in enumerate(zip(
            df['statis_date'],
            df[item_id],
            df['goods_class_name'],
            df['class_name']
        )):
            current_time = pd.Timestamp(current_time)
            try:
                # 过滤时间范围内的记录
                filtered_seq = [
                    s for s in sequence
                    if (current_time - pd.Timestamp(s['oper_time'])).days <= cycle
                ]

                if filtered_seq:
                    seq_array = np.array([
                        (s['sku_id'], s['first_class_name'], s['second_class_name'])
                        for s in filtered_seq
                    ], dtype=object)

                    result_arrays[f'u2i_{cycle}days_{sequence_type}_count'][idx] = np.sum(seq_array[:, 0] == current_item)
                    result_arrays[f'u2i_type1_{cycle}days_{sequence_type}_count'][idx] = np.sum(seq_array[:, 1] == type1)
                    result_arrays[f'u2i_type2_{cycle}days_{sequence_type}_count'][idx] = np.sum(seq_array[:, 2] == type2)
            except Exception as e:
                print(f"Error processing sequence for row {idx}: {e}")

            result_df = pd.DataFrame(result_arrays)
            result_df.index = df.index
            return pd.concat([df, result_df], axis=1)
        else:
            print("序列判断有问题")
            sys.exit()


In [ ]:
def process_features(df, cycle=7, real_clk=False):
    """处理所有类型的交叉特征"""
    result_df = df.copy()

    # 处理实时点击数据
    if real_clk:
        result_df = get_cross_feature(
            result_df,
            sequence_type='click',
            cycle=cycle
        )
        return result_df
    # 处理各类行为特征
    for sequence_type in ['view', 'click', 'purchase']:
        result_df = get_cross_feature(
            result_df,
            sequence_type=sequence_type,
            cycle=cycle
        )
    return result_df

In [ ]:
def process_all_features(df):
    data = process_features(df, cycle=7)
    data = process_features(data, cycle=14)
    data = process_features(data, cycle=30)
    data = process_features(data, cycle=60)
    data = process_features(data, cycle=1, real_clk=True)
    return data

In [ ]:
def generate_sample_data(): 
    # 生成10条样例数据
    data2 = pd.DataFrame({
        # 商品特征
    'statis_date': ['20241201', '20241201', '20241202', '20241202', '20241203', '20241203', '20241204', '20241204', '20241205', '20241205'],
    'skuid': ['1001', '1002', '1003', '1004', '1005', '1006', '1007', '1008', '1009', '1010'],
    'class_name': ['食品', '饮料', '零食', '食品', '饮料', '零食', '食品', '饮料', '零食', '食品'],
    'c_price': [15.9, 25.5, 12.8, 18.5, 22.0, 9.9, 16.5, 28.0, 11.5, 19.9],
    'line_price': [19.9, 29.9, 15.9, 22.9, 25.9, 12.9, 19.9, 32.9, 14.9, 23.9],
    'goods_class_id': ['A001', 'B001', 'C001', 'A002', 'B002', 'C002', 'A003', 'B003', 'C003', 'A004'],
    'goods_class_name': ['速溶咖啡', '果汁', '薯片', '豆腐干', '茶饮', '糖果', '方便面', '碳酸饮料', '饼干', '肉脯'],
    
    # 商品统计特征
    'goods_7day_views': [120, 85, 95, 150, 110, 75, 130, 90, 100, 140],
    'goods_14day_views': [250, 180, 200, 310, 230, 160, 270, 190, 210, 290],
    'goods_30day_views': [520, 380, 420, 650, 480, 340, 560, 400, 440, 600],
    'goods_7day_sales': [45, 30, 35, 55, 40, 25, 48, 32, 38, 50],
    'goods_14day_sales': [95, 65, 75, 115, 85, 55, 100, 70, 80, 105],
    'goods_30day_sales': [200, 140, 160, 240, 180, 120, 210, 150, 170, 220],
    
    # 用户特征
    'mobile': ['user001', 'user002', 'user003', 'user004', 'user005', 'user006', 'user007', 'user008', 'user009', 'user010'],
    'prov_code': ['110000', '310000', '440000', '330000', '510000', '420000', '320000', '370000', '610000', '500000'],
    
    # 用户行为序列（示例）
    'qysc_view_seq': [
        '[{"sku_id":"1001","oper_time":"20241201","is_add_buy":"0","frist_class_name":"饮料","second_class_name":"果汁"}]',
        '[{"sku_id":"1001","oper_time":"20241201","is_add_buy":"0","frist_class_name":"饮料","second_class_name":"果汁"}]',
        '[{"sku_id":"1001","oper_time":"20241201","is_add_buy":"0","frist_class_name":"饮料","second_class_name":"果汁"}]',
        '[{"sku_id":"1001","oper_time":"20241201","is_add_buy":"0","frist_class_name":"饮料","second_class_name":"果汁"}]',
        '[{"sku_id":"1001","oper_time":"20241201","is_add_buy":"0","frist_class_name":"饮料","second_class_name":"果汁"}]',
        '[{"sku_id":"1001","oper_time":"20241201","is_add_buy":"0","frist_class_name":"饮料","second_class_name":"果汁"}]',
        '[{"sku_id":"1001","oper_time":"20241201","is_add_buy":"0","frist_class_name":"饮料","second_class_name":"果汁"}]',
        '[{"sku_id":"1001","oper_time":"20241201","is_add_buy":"0","frist_class_name":"饮料","second_class_name":"果汁"}]',
        '[{"sku_id":"1001","oper_time":"20241201","is_add_buy":"0","frist_class_name":"饮料","second_class_name":"果汁"}]',
        '[{"sku_id":"1001","oper_time":"20241201","is_add_buy":"0","frist_class_name":"饮料","second_class_name":"果汁"}]'
    ],
    
    'qysc_clk_seq': [
        '[{"sku_id":"1001","oper_time":"20241201","is_add_buy":"1","frist_class_name":"食品","second_class_name":"速溶咖啡"}]',
        '[{"sku_id":"1001","oper_time":"20241201","is_add_buy":"1","frist_class_name":"食品","second_class_name":"速溶咖啡"}]',
        '[{"sku_id":"1001","oper_time":"20241201","is_add_buy":"1","frist_class_name":"食品","second_class_name":"速溶咖啡"}]',
        '[{"sku_id":"1001","oper_time":"20241201","is_add_buy":"1","frist_class_name":"食品","second_class_name":"速溶咖啡"}]',
        '[{"sku_id":"1001","oper_time":"20241201","is_add_buy":"1","frist_class_name":"食品","second_class_name":"速溶咖啡"}]',
        '[{"sku_id":"1001","oper_time":"20241201","is_add_buy":"1","frist_class_name":"食品","second_class_name":"速溶咖啡"}]',
        '[{"sku_id":"1001","oper_time":"20241201","is_add_buy":"1","frist_class_name":"食品","second_class_name":"速溶咖啡"}]',
        '[{"sku_id":"1001","oper_time":"20241201","is_add_buy":"1","frist_class_name":"食品","second_class_name":"速溶咖啡"}]',
        '[{"sku_id":"1001","oper_time":"20241201","is_add_buy":"1","frist_class_name":"食品","second_class_name":"速溶咖啡"}]',
        '[{"sku_id":"1001","oper_time":"20241201","is_add_buy":"1","frist_class_name":"食品","second_class_name":"速溶咖啡"}]'
    ],
    
    'qysc_order_seq': [
        '[{"sku_id":"1001","oper_time":"20241201","is_add_buy":"1","frist_class_name":"食品","second_class_name":"速溶咖啡"}]',
        '[{"sku_id":"1001","oper_time":"20241201","is_add_buy":"1","frist_class_name":"食品","second_class_name":"速溶咖啡"}]',
        '[{"sku_id":"1001","oper_time":"20241201","is_add_buy":"1","frist_class_name":"食品","second_class_name":"速溶咖啡"}]',
        '[{"sku_id":"1001","oper_time":"20241201","is_add_buy":"1","frist_class_name":"食品","second_class_name":"速溶咖啡"}]',
        '[{"sku_id":"1001","oper_time":"20241201","is_add_buy":"1","frist_class_name":"食品","second_class_name":"速溶咖啡"}]',
        '[{"sku_id":"1001","oper_time":"20241201","is_add_buy":"1","frist_class_name":"食品","second_class_name":"速溶咖啡"}]',
        '[{"sku_id":"1001","oper_time":"20241201","is_add_buy":"1","frist_class_name":"食品","second_class_name":"速溶咖啡"}]',
        '[{"sku_id":"1001","oper_time":"20241201","is_add_buy":"1","frist_class_name":"食品","second_class_name":"速溶咖啡"}]',
        '[{"sku_id":"1001","oper_time":"20241201","is_add_buy":"1","frist_class_name":"食品","second_class_name":"速溶咖啡"}]',
        '[{"sku_id":"1001","oper_time":"20241201","is_add_buy":"1","frist_class_name":"食品","second_class_name":"速溶咖啡"}]'
    ]
})

    print("样例数据生成完成，数据形状:", data2.shape)
    return data2

In [7]:
def preprocess_sequence(sequence_str, reference_date):
    """预处理序列数据，返回统计结果字典"""
    if sequence_str == '[]' or not sequence_str:
        return {}
    
    sequence = eval(sequence_str)
    stats = {
        7: {'sku': {}, 'type1': {}, 'type2': {}},
        14: {'sku': {}, 'type1': {}, 'type2': {}},
        30: {'sku': {}, 'type1': {}, 'type2': {}},
        60: {'sku': {}, 'type1': {}, 'type2': {}}
    }
    
    reference_date = pd.Timestamp(reference_date)
    
    for record in sequence:
        record_date = pd.Timestamp(record['oper_time'])
        days_diff = (reference_date - record_date).days
        
        sku = record['sku_id']
        type1 = record['frist_class_name']
        type2 = record['second_class_name']
        
        # 更新各个时间窗口的统计值
        for window in [7, 14, 30, 60]:
            if days_diff <= window:
                stats[window]['sku'][sku] = stats[window]['sku'].get(sku, 0) + 1
                stats[window]['type1'][type1] = stats[window]['type1'].get(type1, 0) + 1
                stats[window]['type2'][type2] = stats[window]['type2'].get(type2, 0) + 1
    
    return stats

In [8]:
import numpy as np
import sys 

def get_cross_feature(df, user_id='mobile', item_id='skuid', sequence_type='view', cycle=7):
    """计算用户-商品交叉特征的优化版本"""
    seq_col_map = {
        'view': 'qysc_view_seq',
        'click': 'qysc_clk_seq',
        'purchase': 'qysc_order_seq'
    }
    sequence_col = seq_col_map[sequence_type]
    
    # 检查序列和日期是否都相同
    first_seq = df[sequence_col].iloc[0]
    first_date = df['statis_date'].iloc[0]
    
    if df[sequence_col].eq(first_seq).all() and df['statis_date'].eq(first_date).all():
        # 所有序列和日期都相同，只需计算一次
        stats = preprocess_sequence(first_seq, first_date)
        
        # 创建结果数组
        result_arrays = {
            f'u2i_{cycle}days_{sequence_type}_count': np.zeros(len(df)),
            f'u2i_type1_{cycle}days_{sequence_type}_count': np.zeros(len(df)),
            f'u2i_type2_{cycle}days_{sequence_type}_count': np.zeros(len(df))
        }
        
        # 填充特征值
        for idx, row in df.iterrows():
            if stats:  # 如果有统计数据
                result_arrays[f'u2i_{cycle}days_{sequence_type}_count'][idx] = \
                    stats[cycle]['sku'].get(row[item_id], 0)
                result_arrays[f'u2i_type1_{cycle}days_{sequence_type}_count'][idx] = \
                    stats[cycle]['type1'].get(row['goods_class_name'], 0)
                result_arrays[f'u2i_type2_{cycle}days_{sequence_type}_count'][idx] = \
                    stats[cycle]['type2'].get(row['class_name'], 0)
            else:
                print("统计数据为空")
                sys.exit()
        result_df = pd.DataFrame(result_arrays)
        result_df.index = df.index
        return pd.concat([df, result_df], axis=1)
    else:
        print("序列和日期判断有问题")
        sys.exit()

In [4]:
import pandas as pd 
import numpy as np

def generate_sample_data_v2(): 
    # 生成10条样例数据
    data2 = pd.DataFrame({
        # 商品特征
    'statis_date': ['20241201', '20241201', '20241201', '20241201', '20241201', '20241201', '20241201', '20241201', '20241201', '20241201'],
    'skuid': ['1001', '1002', '1003', '1004', '1005', '1006', '1007', '1008', '1009', '1010'],
    'class_name': ['食品', '饮料', '零食', '食品', '饮料', '零食', '食品', '饮料', '零食', '食品'],
    'c_price': [15.9, 25.5, 12.8, 18.5, 22.0, 9.9, 16.5, 28.0, 11.5, 19.9],
    'line_price': [19.9, 29.9, 15.9, 22.9, 25.9, 12.9, 19.9, 32.9, 14.9, 23.9],
    'goods_class_id': ['A001', 'B001', 'C001', 'A002', 'B002', 'C002', 'A003', 'B003', 'C003', 'A004'],
    'goods_class_name': ['速溶咖啡', '果汁', '薯片', '豆腐干', '茶饮', '糖果', '方便面', '碳酸饮料', '饼干', '肉脯'],
    
    # 商品统计特征
    'goods_7day_views': [120, 85, 95, 150, 110, 75, 130, 90, 100, 140],
    'goods_14day_views': [250, 180, 200, 310, 230, 160, 270, 190, 210, 290],
    'goods_30day_views': [520, 380, 420, 650, 480, 340, 560, 400, 440, 600],
    'goods_7day_sales': [45, 30, 35, 55, 40, 25, 48, 32, 38, 50],
    'goods_14day_sales': [95, 65, 75, 115, 85, 55, 100, 70, 80, 105],
    'goods_30day_sales': [200, 140, 160, 240, 180, 120, 210, 150, 170, 220],
    
    # 用户特征
    'mobile': ['user001', 'user002', 'user003', 'user004', 'user005', 'user006', 'user007', 'user008', 'user009', 'user010'],
    'prov_code': ['110000', '310000', '440000', '330000', '510000', '420000', '320000', '370000', '610000', '500000'],
    
    # 用户行为序列（示例）
    'qysc_view_seq': [
        '[{"sku_id":"1001","oper_time":"20241201","is_add_buy":"0","frist_class_name":"饮料","second_class_name":"果汁"}]',
        '[{"sku_id":"1001","oper_time":"20241201","is_add_buy":"0","frist_class_name":"饮料","second_class_name":"果汁"}]',
        '[{"sku_id":"1001","oper_time":"20241201","is_add_buy":"0","frist_class_name":"饮料","second_class_name":"果汁"}]',
        '[{"sku_id":"1001","oper_time":"20241201","is_add_buy":"0","frist_class_name":"饮料","second_class_name":"果汁"}]',
        '[{"sku_id":"1001","oper_time":"20241201","is_add_buy":"0","frist_class_name":"饮料","second_class_name":"果汁"}]',
        '[{"sku_id":"1001","oper_time":"20241201","is_add_buy":"0","frist_class_name":"饮料","second_class_name":"果汁"}]',
        '[{"sku_id":"1001","oper_time":"20241201","is_add_buy":"0","frist_class_name":"饮料","second_class_name":"果汁"}]',
        '[{"sku_id":"1001","oper_time":"20241201","is_add_buy":"0","frist_class_name":"饮料","second_class_name":"果汁"}]',
        '[{"sku_id":"1001","oper_time":"20241201","is_add_buy":"0","frist_class_name":"饮料","second_class_name":"果汁"}]',
        '[{"sku_id":"1001","oper_time":"20241201","is_add_buy":"0","frist_class_name":"饮料","second_class_name":"果汁"}]'
    ],
    
    'qysc_clk_seq': [
        '[{"sku_id":"1001","oper_time":"20241201","is_add_buy":"1","frist_class_name":"食品","second_class_name":"速溶咖啡"}]',
        '[{"sku_id":"1001","oper_time":"20241201","is_add_buy":"1","frist_class_name":"食品","second_class_name":"速溶咖啡"}]',
        '[{"sku_id":"1001","oper_time":"20241201","is_add_buy":"1","frist_class_name":"食品","second_class_name":"速溶咖啡"}]',
        '[{"sku_id":"1001","oper_time":"20241201","is_add_buy":"1","frist_class_name":"食品","second_class_name":"速溶咖啡"}]',
        '[{"sku_id":"1001","oper_time":"20241201","is_add_buy":"1","frist_class_name":"食品","second_class_name":"速溶咖啡"}]',
        '[{"sku_id":"1001","oper_time":"20241201","is_add_buy":"1","frist_class_name":"食品","second_class_name":"速溶咖啡"}]',
        '[{"sku_id":"1001","oper_time":"20241201","is_add_buy":"1","frist_class_name":"食品","second_class_name":"速溶咖啡"}]',
        '[{"sku_id":"1001","oper_time":"20241201","is_add_buy":"1","frist_class_name":"食品","second_class_name":"速溶咖啡"}]',
        '[{"sku_id":"1001","oper_time":"20241201","is_add_buy":"1","frist_class_name":"食品","second_class_name":"速溶咖啡"}]',
        '[{"sku_id":"1001","oper_time":"20241201","is_add_buy":"1","frist_class_name":"食品","second_class_name":"速溶咖啡"}]'
    ],
    
    'qysc_order_seq': [
        '[{"sku_id":"1001","oper_time":"20241201","is_add_buy":"1","frist_class_name":"食品","second_class_name":"速溶咖啡"}]',
        '[{"sku_id":"1001","oper_time":"20241201","is_add_buy":"1","frist_class_name":"食品","second_class_name":"速溶咖啡"}]',
        '[{"sku_id":"1001","oper_time":"20241201","is_add_buy":"1","frist_class_name":"食品","second_class_name":"速溶咖啡"}]',
        '[{"sku_id":"1001","oper_time":"20241201","is_add_buy":"1","frist_class_name":"食品","second_class_name":"速溶咖啡"}]',
        '[{"sku_id":"1001","oper_time":"20241201","is_add_buy":"1","frist_class_name":"食品","second_class_name":"速溶咖啡"}]',
        '[{"sku_id":"1001","oper_time":"20241201","is_add_buy":"1","frist_class_name":"食品","second_class_name":"速溶咖啡"}]',
        '[{"sku_id":"1001","oper_time":"20241201","is_add_buy":"1","frist_class_name":"食品","second_class_name":"速溶咖啡"}]',
        '[{"sku_id":"1001","oper_time":"20241201","is_add_buy":"1","frist_class_name":"食品","second_class_name":"速溶咖啡"}]',
        '[{"sku_id":"1001","oper_time":"20241201","is_add_buy":"1","frist_class_name":"食品","second_class_name":"速溶咖啡"}]',
        '[{"sku_id":"1001","oper_time":"20241201","is_add_buy":"1","frist_class_name":"食品","second_class_name":"速溶咖啡"}]'
    ]
})

    print("样例数据生成完成，数据形状:", data2.shape)
    return data2

In [5]:
data = generate_sample_data_v2()

样例数据生成完成，数据形状: (10, 18)


In [6]:
data.head()

,statis_date,skuid,class_name,c_price,line_price,goods_class_id,goods_class_name,goods_7day_views,goods_14day_views,goods_30day_views,goods_7day_sales,goods_14day_sales,goods_30day_sales,mobile,prov_code,qysc_view_seq,qysc_clk_seq,qysc_order_seq
0,20241201,1001,食品,15.9,19.9,A001,速溶咖啡,120,250,520,45,95,200,user001,110000,"[{""sku_id"":""1001"",""oper_time"":""20241201"",""is_a...","[{""sku_id"":""1001"",""oper_time"":""20241201"",""is_a...","[{""sku_id"":""1001"",""oper_time"":""20241201"",""is_a..."
1,20241201,1002,饮料,25.5,29.9,B001,果汁,85,180,380,30,65,140,user002,310000,"[{""sku_id"":""1001"",""oper_time"":""20241201"",""is_a...","[{""sku_id"":""1001"",""oper_time"":""20241201"",""is_a...","[{""sku_id"":""1001"",""oper_time"":""20241201"",""is_a..."
2,20241201,1003,零食,12.8,15.9,C001,薯片,95,200,420,35,75,160,user003,440000,"[{""sku_id"":""1001"",""oper_time"":""20241201"",""is_a...","[{""sku_id"":""1001"",""oper_time"":""20241201"",""is_a...","[{""sku_id"":""1001"",""oper_time"":""20241201"",""is_a..."
3,20241201,1004,食品,18.5,22.9,A002,豆腐干,150,310,650,55,115,240,user004,330000,"[{""sku_id"":""1001"",""oper_time"":""20241201"",""is_a...","[{""sku_id"":""1001"",""oper_time"":""20241201"",""is_a...","[{""sku_id"":""1001"",""oper_time"":""20241201"",""is_a..."
4,20241201,1005,饮料,22.0,25.9,B002,茶饮,110,230,480,40,85,180,user005,510000,"[{""sku_id"":""1001"",""oper_time"":""20241201"",""is_a...","[{""sku_id"":""1001"",""oper_time"":""20241201"",""is_a...","[{""sku_id"":""1001"",""oper_time"":""20241201"",""is_a..."


In [9]:
res = get_cross_feature(data, sequence_type='view', cycle=7)

In [10]:
res.head()

,statis_date,skuid,class_name,c_price,line_price,goods_class_id,goods_class_name,goods_7day_views,goods_14day_views,goods_30day_views,...,goods_14day_sales,goods_30day_sales,mobile,prov_code,qysc_view_seq,qysc_clk_seq,qysc_order_seq,u2i_7days_view_count,u2i_type1_7days_view_count,u2i_type2_7days_view_count
0,20241201,1001,食品,15.9,19.9,A001,速溶咖啡,120,250,520,...,95,200,user001,110000,"[{""sku_id"":""1001"",""oper_time"":""20241201"",""is_a...","[{""sku_id"":""1001"",""oper_time"":""20241201"",""is_a...","[{""sku_id"":""1001"",""oper_time"":""20241201"",""is_a...",1.0,0.0,0.0
1,20241201,1002,饮料,25.5,29.9,B001,果汁,85,180,380,...,65,140,user002,310000,"[{""sku_id"":""1001"",""oper_time"":""20241201"",""is_a...","[{""sku_id"":""1001"",""oper_time"":""20241201"",""is_a...","[{""sku_id"":""1001"",""oper_time"":""20241201"",""is_a...",0.0,0.0,0.0
2,20241201,1003,零食,12.8,15.9,C001,薯片,95,200,420,...,75,160,user003,440000,"[{""sku_id"":""1001"",""oper_time"":""20241201"",""is_a...","[{""sku_id"":""1001"",""oper_time"":""20241201"",""is_a...","[{""sku_id"":""1001"",""oper_time"":""20241201"",""is_a...",0.0,0.0,0.0
3,20241201,1004,食品,18.5,22.9,A002,豆腐干,150,310,650,...,115,240,user004,330000,"[{""sku_id"":""1001"",""oper_time"":""20241201"",""is_a...","[{""sku_id"":""1001"",""oper_time"":""20241201"",""is_a...","[{""sku_id"":""1001"",""oper_time"":""20241201"",""is_a...",0.0,0.0,0.0
4,20241201,1005,饮料,22.0,25.9,B002,茶饮,110,230,480,...,85,180,user005,510000,"[{""sku_id"":""1001"",""oper_time"":""20241201"",""is_a...","[{""sku_id"":""1001"",""oper_time"":""20241201"",""is_a...","[{""sku_id"":""1001"",""oper_time"":""20241201"",""is_a...",0.0,0.0,0.0
